# Principles of Digital Communication

## To do : 
- Avec le simple repetition code (pas de parity check), avec un message de 10 charactères, déterminer le nombre de répétitions à partir duquel sur 100 runs, aucune erreur n'est obtenue. Super important : c'est la baseline qu'on va utiliser comme nombre pour parity_incrust sur le second point.
- Avec le code hybride : jouer avec les paramètres k, block_length, parity_incrust, batch_size ( avec batch_size == block_length), pour déterminer les paramètres optimaux pour que sur 100 runs, il n'y ait pas d'erreurs. Je doute que ce nombre vaille 0, mais bon, c'est de l'optimisation, c'est vous les data scientists :) . 
- En réalité, le code "hybride" est une variante du LDPC (une méthode d'encodage qui est utilisée dans le standard de la 5G) que j'ai essayé d'implémenter, mais qui n'est pas optimale. Pour faire simple, mon code hybride utilise 1 parity bit par set disjoint de 8 bits d'information, l'optimisation du LDPC consiste à rajouter des bits de protection sur des set non-disjoints. Par exemple : imaginez que vous avez 24 bits d'information à protéger, le code hybride va avoir un bit de parity après le 8ème bit, un bit de parity après le 16ème bit, et un bit de parity après le 24ème bit. Le LDPC va rajouter un bit de parity qui couvre les bits 1 à 8, puis un bit de parity qui couvre les bits 7 à 14, puis un bit de parity qui couvre les bits 12 à 19, puis un bit de parity qui couvre les bits 17 à 24. En gros, les bits de parity couvrent des "overlapping sets", mais faut en rajouter. Je soupçonne que si c'est bien implémenté, on peut atteindre un rate beaucoup plus bas, en ayant besoin de moins de répétitions, faut l'implémenter.
- Explorer ce qu'il est possible de faire avec d'autres types de codes : si vous pouvez explorer les "Tornado Codes" (qui sont , "Reed-Solomon codes", "Turbo Codes" et "Reed-Muller Codes", ce serait top! En fait, il faut savoir que le code sur lequel on a travaillé dans le homework 2 (le Hadamard Code), est en fait un cas particulier d'un Reed-Muller code. Vous pouvez partir de là.

### Imports

In [177]:
import helpers
import numpy as np
from scipy.linalg import hadamard

# Encoding 

In [178]:
input = "input.txt"
output = "output.txt"


In [179]:
# Returns a list of characters from a given textfile
text = helpers.read_text_file(input)

# Convert text to binary
text_binary = helpers.chars_to_bits_partial(text)

# Convert binary text to -1 and 1's, useful later for comparison
text_for_compare = helpers.from_zero_to_neg(text_binary)


In [180]:
# Divide our 640 bits message into 64 chunks of size 10 
chunks = []
for i in range(0,len(text_binary)-10,10):
    chunks.append(text_binary[i:i+10])
    
last_chunk = []
last_chunk.extend(text_binary[-10:-1])

last_bit = text_binary[-1]
print(len(chunks))
print(len(chunks[0]))


63
10


In [181]:
# Construct Hadamard Matrix
hadam = hadamard(512)
hadam_2 = hadamard(256)

In [182]:
# Convert our chunks to a base 10 representation
base10_chunks = []
for chunk in chunks:
    base10_repr = helpers.to_base10(chunk)
    base10_chunks.append(base10_repr)

base10_lastchunk = helpers.to_base10(last_chunk)
print(len(base10_chunks))
print(base10_lastchunk)

63
36


In [183]:
# Create message according to hadamard matrix
encoded_message = []
for c in base10_chunks:
    if c < 512:
        encoded_message.append(hadam[c])
    else :
        encoded_message.append(-hadam[c-512])

c = base10_lastchunk
if c < 256 : 
    encoded_message.append(hadam_2[c])
else : 
    encoded_message.append(-hadam_2[c-256])
        
last_bit_rep = []
for i in range(120):
    last_bit_rep.append(last_bit)

encoded_message.append(last_bit_rep)
encoded_message_flat = [item for sublist in encoded_message for item in sublist]



In [184]:
len(encoded_message_flat)

32632

In [185]:
# Save file 
np.savetxt("encoded.txt",encoded_message_flat,fmt='%i')

### Sending to server

In [186]:
%run client.py --input_file="encoded.txt" --output_file="received.txt" --srv_hostname=iscsrv72.epfl.ch --srv_port=80

# Decoding

In [187]:
# Read output file
output = open('received.txt', 'r').read().splitlines()

In [188]:
# Convert output from char to float 
output = [float(i) for i in output]

In [189]:
output = np.asarray(output)

In [190]:
hadam = np.asarray(hadam)
hadam_2 = np.asarray(hadam_2)

In [191]:
# Divide our message into 64 chunks of size 512
output_chunks = []
for i in range(0,len(output)-256-120,512):
    output_chunks.append(output[i:i+512])

In [192]:
output_chunks = np.asarray(output_chunks)

In [193]:
# Create U matrix for decoding
U = []
for c in output_chunks :
    U.append(np.matmul(hadam,c))
U = np.asarray(U)
    


In [194]:
# Get index of max of each line:
U_max_indices = []
for line in U:
    U_max_indices.append(helpers.get_max_index(line))
U_max_indices = np.asarray(U_max_indices)

In [195]:
# Decode 
U_max_indices_binary = []
for i in range(len(U_max_indices)):
    max_index = U_max_indices[i]
    if(U[i,max_index]<0):
        for c in "{0:010b}".format(max_index+512):
            U_max_indices_binary.append(int(c))
    else:
        for c in "{0:010b}".format(max_index):
                U_max_indices_binary.append(int(c))


In [196]:
#decode_last_bloc_9 (1)

last_bloc9 = output[-376:-120]
last_bloc9 = np.asarray(last_bloc9)
U2 = []
U2.append(np.matmul(hadam_2,last_bloc9))
U2 = np.asarray(U2)
U2_max_index = []
for line in U2 : 
    U2_max_index.append(helpers.get_max_index(line))
U2_max_index = np.asarray(U2_max_index)





In [197]:
#decode last bloc 9 (2)
U2_max_index_binary = [] 
max_index_2 = U2_max_index[0]
if U2[0,max_index_2] < 0 : 
    for c in "{0:09b}".format(max_index_2+256):
        U2_max_index_binary.append(int(c))
else : 
    for c in "{0:09b}".format(max_index_2):
        U2_max_index_binary.append(int(c))
        
        

    

In [198]:
U2_max_index_binary

[0, 0, 0, 1, 0, 0, 1, 0, 0]

In [199]:
#decode last bit
last_bit_rep = output[-120:]
acc = 0
last_bit = 0
for i in range(len(last_bit_rep)):
    acc += last_bit_rep[i]

if acc > 0 : 
    last_bit = 1
    

In [200]:
#combine all 

for b in range(len(U2_max_index_binary)):
    U_max_indices_binary.append(U2_max_index_binary[b])
U_max_indices_binary.append(last_bit)


In [201]:
len(U_max_indices_binary)

640

In [202]:
helpers.compute_Hamming(U_max_indices_binary,text_binary)

(0, [])

In [203]:
helpers.from_bits_to_chars(U_max_indices_binary)

'FtIWW9JH2+XjOJfexiSouSOWPLdyVGHDm4ZjnlCgLb6zdRnTXd08tv1F+C+CDFdwlNMFCMx+bYe0gmLI'